In [ ]:
import numpy as np
from PybulletCore import pybullet_core
%load_ext autoreload
%autoreload 2

## 1. Open Pybullet GUI

In [ ]:
pb = pybullet_core()
# pb.connect_pybullet(robot_name = "Indy7", joint_limit=False)
pb.connect_pybullet(robot_name = "IndyRP2", joint_limit=False)

## 2. Move robot

In [ ]:
pb.MoveRobot(2*(1-2*np.random.rand(7)), verbose=True)

## 3. Close Pybullet GUI

In [ ]:
pb.disconnect_pybullet()